### ДЗ к Уроку 3

## Сделать RFM-анализ на основе данных по продажам за 2 года (из предыдущего дз).

1. Определить критерии для каждой буквы R, F, M (т.е. к примеру, R – 3 для клиентов, которые покупали <= 30 дней от последней даты в базе, R – 2 для клиентов, которые покупали > 30 и менее 60 дней от последней даты в базе и т.д.)
2. Для каждого пользователя получить набор из 3 цифр (от 111 до 333, где 333 – самые классные пользователи)
3. Ввести группировку, к примеру, 333 и 233 – это Vip, 1XX – это Lost, остальные Regular ( можете ввести боле глубокую сегментацию)
4. Для каждой группы из п. 3 найти количество пользователей, которые попали в них и % товарооборота, которое они сделали на эти 2 года.
5. Проверить, что общее количество пользователей бьется с суммой количества пользователей по группам из п. 3 (если у вас есть логические ошибки в создании групп, у вас не собьются цифры). То же самое сделать и по деньгам.

!!! Результаты присылаем скрином или в пдф (где можно увидеть название группы, правила попадания в группу, кол-во пользователей, их товарооборот, а так же процент по кол-ву пользвателей и по деньгам)

### 0. Подготовка данных

In [1]:
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('D:/GeekBrains/Database for analytics/orders_20190822.csv', sep=";", decimal=",", parse_dates=["o_date"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
id_o       int64
user_id    int64
price      float64
o_date     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 61.1 MB


In [4]:
data = df.copy()

In [5]:
data.head(5)

,id_o,user_id,price,o_date
0,1234491,337544,539.0,2016-01-01
1,1234494,171642,153.3,2016-01-01
2,1234497,260596,55.3,2016-01-01
3,1234498,1105609,752.5,2016-01-01
4,1234500,982696,4410.0,2016-01-01


определяем текущий момент времени относительно которого будем делать анализ, в данном случае текущий момент будет равен: день последней транзакции + 1 день

In [6]:
snapshot_date = max(data['o_date']) + dt.timedelta(days=1)

группируем данные по ID покупателя, при этом вычисляем дату последней покупки (Recency), количество покупок данного покупателя за весь период (Frequency) и общую сумму покупок данного покупателя (Monetary)

In [7]:
df_RFM = data.groupby(['user_id']).agg({
    'o_date': lambda x: (snapshot_date - x.max()).days,
    'id_o': 'count',
    'price': 'sum'})

df_RFM.rename(columns = { 'o_date': 'Recency',
                            'id_o': 'Frequency',
                            'price': 'Monetary'}, inplace=True)

In [8]:
df_RFM.head()

,Recency,Frequency,Monetary
user_id,,,
0,507,1,494.2
1,358,2,13844.6
76,112,2,1863.4
90,12,4,6561.8
91,52,4,5413.1


### 1. Определить критерии для каждой буквы R, F, M 
(т.е. к примеру, R – 3 для клиентов, которые покупали <= 30 дней от последней даты в базе, R – 2 для клиентов, которые покупали > 30 и менее 60 дней от последней даты в базе и т.д.)

Recency: 1 - появлялся больше 1 года назад (x >= 366 дней), 2 - появлялся от полугода до года (180 <= x < 366), 3 - появлялся до полугода (45 <= x < 180), 4 - появлялся недавно (x < 45)

Frequency: 1 - разовая покупка (x <= 1), 2 - несколько покупок (1 < x <= 24), 3 - регулярные покупки (24 < x <= 150), 4 - постоянные покупки (x < 150)

Monetary: 1 - мизерная сумма (x <= 3000), 2 -  (3000 < x <= 60000), 3 -  (60000 < x <= 200000), 4 -  (x > 200000)

In [9]:
datamart = df_RFM.copy()

In [10]:
datamart.head()

,Recency,Frequency,Monetary
user_id,,,
0,507,1,494.2
1,358,2,13844.6
76,112,2,1863.4
90,12,4,6561.8
91,52,4,5413.1


In [11]:
def Recency(df):
    if df['Recency'] >= 366:
        return 1
    elif ((df['Recency'] >= 180) & (df['Recency'] < 366)):
        return 2
    elif ((df['Recency'] >= 45) & (df['Recency'] < 180)):
        return 3        
    else:
        return 4 
        
def Frequency(df):
    if df['Frequency'] <= 1:
        return 1
    elif ((df['Frequency'] > 1) & (df['Frequency'] <= 24)):
        return 2
    elif ((df['Frequency'] > 24) & (df['Frequency'] <= 150)):
        return 3       
    else:
        return 4

def Monetary(df):
    if df['Monetary'] <= 3000:
        return 1
    elif ((df['Monetary'] > 3000) & (df['Monetary'] <= 60000)):
        return 2
    elif ((df['Monetary'] > 60000) & (df['Monetary'] <= 200000)):
        return 3        
    else:
        return 4      

In [12]:
def rfm_criteria(df):
    df['R'] = df.apply(Recency, axis='columns')
    df['F'] = df.apply(Frequency, axis='columns')
    df['M'] = df.apply(Monetary, axis='columns')
    return df

In [13]:
rfm_criteria(datamart)

,Recency,Frequency,Monetary,R,F,M
user_id,,,,,,
0,507,1,494.20,1,1,1
1,358,2,13844.60,2,2,2
76,112,2,1863.40,3,2,1
90,12,4,6561.80,4,2,2
91,52,4,5413.10,3,2,2
95,16,4,3410.40,4,2,2
155,423,2,6745.20,1,2,2
160,458,1,609.00,1,1,1
161,524,2,14697.20,1,2,2


### 2. Для каждого пользователя получить набор из 3 цифр

In [18]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

In [19]:
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1)

In [20]:
datamart.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment
user_id,,,,,,,
0,507,1,494.2,1,1,1,111
1,358,2,13844.6,2,2,2,222
76,112,2,1863.4,3,2,1,321
90,12,4,6561.8,4,2,2,422
91,52,4,5413.1,3,2,2,322


### 2. Ввести группировку
к примеру, 333 и 233 – это Vip, 1XX – это Lost, остальные Regular ( можете ввести боле глубокую сегментацию)

In [22]:
datamart['RFM_Segment'] = datamart['RFM_Segment'].astype(int)

In [23]:
def rfm_group(df):
    if df['RFM_Segment'] in [444, 443, 344, 343]: 
        return 'super client'
    elif df['RFM_Segment'] in [434, 433, 424, 423, 334, 333, 324, 323]: 
        return 'potental good client'  
    elif df['RFM_Segment'] in [414, 413, 412, 411, 314, 313, 312, 311]:   
        return 'new client'
    elif df['RFM_Segment'] in [244, 243, 144, 143, 234, 233, 134, 133, 224, 223, 124, 123]:
        return 'lost good client'      
    elif df['RFM_Segment'] in [114, 113, 112, 111, 214, 213, 212, 211]: 
        return 'lost new client'    
    else:
        return 'undescribed client' 

In [24]:
datamart['RFM_group'] = datamart.apply(rfm_group, axis=1)

### 4. Для каждой группы из п. 3 найти количество пользователей, которые попали в них и % товарооборота, которое они сделали на эти 2 года.

In [25]:
total_sum = df['price'].sum()
total_sum

4542687441.25

In [26]:
rfm_level_agg = datamart.groupby('RFM_group').agg({
    'RFM_group': 'count',
    'Monetary': 'sum'
}).round(2)

In [27]:
rfm_level_agg['% user'] = (rfm_level_agg['RFM_group']*100/len(df['user_id'].unique())).round(2)

In [28]:
rfm_level_agg['% monetary'] = (rfm_level_agg['Monetary']*100/total_sum).round(2)

In [29]:
rfm_level_agg.rename(columns = { 'RFM_group': 'users'})

,users,Monetary,% user,% monetary
RFM_group,,,,
lost good client,571,7.395062e+07,0.06,1.63
lost new client,498956,1.135873e+09,49.15,25.00
new client,275541,6.507064e+08,27.14,14.32
potental good client,3737,3.803785e+08,0.37,8.37
super client,93,1.535584e+08,0.01,3.38
undescribed client,236221,2.148221e+09,23.27,47.29


### 5. Проверить, что общее количество пользователей бьется с суммой количества пользователей по группам из п. 3 (если у вас есть логические ошибки в создании групп, у вас не собьются цифры). То же самое сделать и по деньгам.

In [30]:
# проверим верность количества пользователей
rfm_level_agg['RFM_group'].sum() == len(df['user_id'].unique())

True

In [31]:
# проверим верность суммы товарооборота
rfm_level_agg['Monetary'].sum() == total_sum

True